In [1]:
import os
import numpy as np
import pandas as pd
import rasterio as rio
import matplotlib.pyplot as plt
import xarray as xr
import datetime as dt
import rioxarray
import seaborn as sns
import geopandas as gpd

In [2]:
# function to rewrite coordinates from metadata
def coord_range(ds):
    latrange = np.linspace(float(ds.attrs['Y_FIRST']),
                           ((float(ds.attrs['Y_STEP'])*float(ds.attrs['LENGTH']))+float(ds.attrs['Y_FIRST'])),
                           int(ds.attrs['LENGTH']))
    lonrange = np.linspace(float(ds.attrs['X_FIRST']),
                           ((float(ds.attrs['X_STEP'])*float(ds.attrs['WIDTH']))+float(ds.attrs['X_FIRST'])),
                           int(ds.attrs['WIDTH']))
    return latrange, lonrange

def mintpy2d_to_xarray(fn, crs):
    ds = xr.open_dataset(fn)
    ds = ds.rename_dims({'phony_dim_0':'y',
                         'phony_dim_1':'x'
                        })
    latrange, lonrange = coord_range(ds)
    ds = ds.assign_coords({'y': ('y', latrange),
                           'x': ('x', lonrange)})
    ds = ds.rio.write_crs(crs)
    
    return ds

In [3]:
orbit = 'DT56'
year = '2017'
frame = 'frame_3'

In [4]:
# function to clip files and run mintpy for multiple years 
def save_veloc_multiyear(orbit, year_list, frame_list, crs=32613):
    # hardcoded paths for now 
    home_path = '/mnt/c/Users/qbren/Desktop/taco/projects/indennt/proc/data'
    for frame in frame_list:
        print(f'working on {frame}')
        for year in year_list:
            print(f'working on {year}')
            mintpy_path = f'{home_path}/signal_mintpy/{orbit}/{frame}/mintpy_{year}'

            vel_ds = mintpy2d_to_xarray(f'{mintpy_path}/velocity.h5', crs)
            vel_ds.velocity.rio.to_raster(f'{home_path}/signal_maps/{orbit}_{frame}_{year}_signal.tif')

In [5]:
save_veloc_multiyear(orbit, [year], [frame])

working on frame_3
working on 2017
